In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
import scipy.io as sio
import json

In [3]:
mat_path = 'MPHB-label-mat/MPHB-label.mat'
data = sio.loadmat(mat_path, struct_as_record=False)['GT']

In [4]:
idx_arr = []
bbox_arr = []
filename_arr = []
image_type_arr = []

In [5]:
for item in tqdm(data[0]):
    num_bboxes = item.__dict__['bbox'].shape[0]
    idx = item.__dict__['idx'][0]
    source = item.__dict__['source'][0].split(' ')
    
    image_type = source[0][:-1]
    filename = source[1]
    
    for i in range(num_bboxes):
        idx_arr.append(idx)
        bbox_arr.append(item.__dict__['bbox'][i])
        image_type_arr.append(image_type)
        filename_arr.append(filename)

  0%|          | 0/26675 [00:00<?, ?it/s]

In [6]:
bbox_arr = np.array(bbox_arr)
bbox_arr

array([[467.96543779,  84.73963134, 591.83640553, 476.99769585],
       [521.05299539, 120.13133641, 712.75806452, 606.76728111],
       [358.84101382,  78.84101382, 512.20506912, 612.66589862],
       ...,
       [ 20.        ,   5.        , 143.        , 136.        ],
       [  6.        ,  12.        ,  61.        , 161.        ],
       [ 13.        ,   1.        ,  56.        , 187.        ]])

In [7]:
print(len(idx_arr))
print(bbox_arr.shape)

29732
(29732, 4)


In [8]:
data[0, 0].__dict__

{'_fieldnames': ['idx', 'bbox', 'source'],
 'idx': array(['00001'], dtype='<U5'),
 'bbox': array([[467.96543779,  84.73963134, 591.83640553, 476.99769585],
        [521.05299539, 120.13133641, 712.75806452, 606.76728111]]),
 'source': array(['MPII: 000001163.jpg'], dtype='<U19')}

In [9]:
df = pd.DataFrame({'idx': idx_arr, 'filename': filename_arr, 'type': image_type_arr, 'xmin': bbox_arr[:, 0], 'xmax': bbox_arr[:, 2], 'ymin': bbox_arr[:, 1], 'ymax': bbox_arr[:, 3]})

In [15]:
with open('index_to_class.json', 'r') as f:
    d_map = json.load(f)

label_map  = {'standing': 0, 'bent': 4, 'kneeling': 3, 'lying': 2, 'sitting': 1, 'occlusion': 5}

df['class'] = df['idx'].map(lambda x: d_map[x])
df['label'] = df['class'].map(lambda x: label_map[x])

In [16]:
df.head()

,idx,filename,type,xmin,xmax,ymin,ymax,class,label
0,00001,000001163.jpg,MPII,467.965438,591.836406,84.739631,476.997696,standing,0
1,00001,000001163.jpg,MPII,521.052995,712.758065,120.131336,606.767281,standing,0
2,00002,000003072.jpg,MPII,358.841014,512.205069,78.841014,612.665899,standing,0
3,00002,000003072.jpg,MPII,591.836406,898.564516,123.080645,597.919355,standing,0
4,00003,000004812.jpg,MPII,1312.205069,1825.384793,471.928571,1069.163594,bent,4


In [17]:
df.to_csv('mphb_datasets.csv', index=False)